<a href="https://colab.research.google.com/github/Joel-Vijo/Neural-Machine-Translation/blob/main/Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 1.2 MB 56.9 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0


In [2]:
from google.colab import drive
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import nltk
import numpy as np
import random
import spacy
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator
from torchtext.data.metrics import bleu_score

In [3]:
device = torch.device("cuda" )
print("Notebook is running on", device)
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Notebook is running on cuda


In [6]:
!python -m spacy download de
!python -m spacy download en
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.3.0/de_core_news_sm-3.3.0-py3-none-any.whl (14.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.3.0/en_core_web_sm-3.3.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [8]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [9]:
!ls -al .data/multi30k

ls: cannot access '.data/multi30k': No such file or directory


In [11]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG),root=".data")

In [12]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [13]:
class Encoder(nn.Module):
    def __init__(self,input_size,enc_hidden_size,dec_hidden_size,embedding_dim):
        super(Encoder,self).__init__()
        self.embed=nn.Embedding(input_size,embedding_dim)
        self.rnn=nn.RNN(embedding_dim,enc_hidden_size,bidirectional=True)
        self.h2o=nn.Linear(enc_hidden_size*2,dec_hidden_size)
    def forward(self,input,hidden):
        input=self.embed(input).to(device)
        output,hidden=self.rnn(input)
        hidden=torch.tanh(self.h2o(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        return output,hidden

In [14]:
class Attention(nn.Module):
    def __init__(self,enc_hidden_size,dec_hidden_size):
        super(Attention,self).__init__()
        self.att=nn.Linear(enc_hidden_size*2+dec_hidden_size,dec_hidden_size)
        self.a2o=nn.Linear(dec_hidden_size,1,bias=False)
    def forward(self,encoding_output,hidden):
        hidden = hidden.unsqueeze(1).repeat(1, encoding_output.shape[0], 1)
        encoding_output = encoding_output.permute(1, 0, 2)
        attent_concat=torch.cat((hidden,encoding_output),dim=2)
        e=self.att(attent_concat)
        output=self.a2o(e)
        output=output.squeeze(2)
        output=F.softmax(output,dim=1)
        return output         

In [15]:
class Decoder(nn.Module):
    def __init__(self,output_size,enc_hidden_size,dec_hidden_size,embedding_dim,attention):
        super(Decoder,self).__init__()
        self.output_size=output_size
        self.attention=attention
        self.embed=nn.Embedding(output_size,embedding_dim)
        self.rnn=nn.RNN(enc_hidden_size*2+embedding_dim,dec_hidden_size)
        self.h2o=nn.Linear(enc_hidden_size*2+embedding_dim+dec_hidden_size,output_size)
        
    def forward(self,input,hidden,encoder_outputs):
        #print("Input",input.size())
        input=self.embed(input)
        a=self.attention(encoder_outputs,hidden)
        a=a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        w=torch.bmm(a,encoder_outputs)
        w=w.permute(1,0,2)
        hidden=hidden.unsqueeze(0)
        output,hidden=self.rnn(torch.cat((w,input),dim=2),hidden)
        #print(output.size())
        input=input.squeeze(0)
        w=w.squeeze(0)
        output=output.squeeze(0)
        output=self.h2o(torch.cat((output,w,input),dim=1))
        #print("Output",output.size())
        return output,hidden

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.enc=encoder
        self.dec=decoder
    def forward(self, hidden_size, source, target, teacher_forcing_ratio=0.5):
        batch_size=target.shape[1]
        target_length=target.shape[0]
        vocab_size=self.dec.output_size
        outputs=torch.zeros(target_length,batch_size,vocab_size).to(device)
        inputs=target[0,:]
        inputs=inputs.unsqueeze(0)
        hidden=(torch.zeros((1, batch_size, hidden_size)),torch.zeros((1, batch_size, hidden_size)))
        encoder_outputs,hidden=self.enc(source,hidden)
        #print(hidden.size())
        for i in range(1,target_length):
          output,hidden=self.dec(inputs,hidden,encoder_outputs)
          #print("Done")
          #print(hidden.size())
          hidden=hidden.squeeze(0)
          #print(hidden.size())
          outputs[i]=output
          teacher_force=random.random()<teacher_forcing_ratio
          if(teacher_force):
            inputs=target[i]
            #print("Input1 ",inputs.size())
            inputs=inputs.unsqueeze(0)
          else:
            inputs=output.argmax(1)
            #print("Input2",inputs.size())
            inputs=inputs.unsqueeze(0)
        return outputs

In [40]:
i_size=len(SRC.vocab)
o_size=len(TRG.vocab)
e_embed_dim=256
d_embed_dim=256
e_h_size=512
d_h_size=512
learning_rate=0.001
enc=Encoder(i_size,e_h_size,d_h_size,e_embed_dim)
att=Attention(e_h_size,d_h_size)
dec=Decoder(o_size,e_h_size,d_h_size,d_embed_dim,att)
model=Seq2Seq(enc,dec).to(device)
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
optimizer = torch.optim.AdamW(model.parameters(),learning_rate)
BATCH_SIZE = 128
print(o_size)
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,device=device)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

5893
The model has 17,104,389 trainable parameters


In [41]:
def train(model,hidden_size,iterator,criterion,optimiser):
  model.train()
  epoch_loss=0
  for i,batch in enumerate(iterator):
    src = batch.src
    trg = batch.trg    
    optimizer.zero_grad()
    output = model(hidden_size, src, trg)
    o_size=output.shape[-1]
    output = output[1:].view(-1, o_size)
    trg = trg[1:].view(-1)
    loss = criterion(output, trg)   
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss/len(iterator)

In [42]:
for i in range(20):
  train_loss=train(model,e_h_size,train_iterator,criterion,optimizer)
  print("Training",train_loss)

Training 4.424807555875064
Training 3.34727902139336
Training 2.8558163180750373
Training 2.524192287008143
Training 2.2898680696403404
Training 2.0754465788996694
Training 1.8771815599323896
Training 1.7876382399235529
Training 1.6522211041219435
Training 1.5386886980039958
Training 1.414779695645303
Training 1.3240941334926084
Training 1.2548796802365307
Training 1.1723217155439738
Training 1.092545553188492
Training 1.034842373516066
Training 0.9628520794376928
Training 0.8869004344099943
Training 0.8131342992383478
Training 0.7540306813916445


In [43]:
def ipTensor(sentence, src_field):
    if isinstance(sentence, list):
        tokens = [src_field.init_token] + [token.lower() for token in sentence] + [src_field.eos_token]
    else:
        tokens = [src_field.init_token] + tokenize_de(sentence) + [src_field.eos_token]
    seq_len = len(tokens)
    ip_tensor = torch.LongTensor([src_field.vocab.stoi[token] for token in tokens]).to(device)
    return ip_tensor.view(seq_len, 1)

In [44]:
def Evaluate(iterator, model, criterion):
    model.eval()
    eval_loss = 0
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            model.zero_grad()
            source = batch.src
            target = batch.trg
            print(target)
            outputs = model(e_h_size,source, target)
            outputs = outputs[1:].view(-1, o_size)
            targets = target[1:].view(-1)
            batch_loss = criterion(outputs, targets)
            eval_loss += batch_loss.item()
        
        return eval_loss/len(iterator)

In [45]:
model.eval()
test_loss = Evaluate(test_iterator, model, criterion)
print(test_loss)

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  16,  110,    4,  ...,    4,   24,   16],
        [1909,   19,   34,  ...,   14,   14,   30],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')
tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [ 14,   4,   4,  ...,   4,  63, 795],
        [395,  35,   9,  ...,  14, 270,   6],
        ...,
        [  3,   3,   3,  ...,   3,   3,   3],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   4,    4,   63,  ...,    7,   25,    7],
        [   9,   38,  929,  ...,   34, 1547, 2222],
        ...,
        [   5,    5,    5,  ...,    1,    1,    1],
        [   3,    3,    3,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')
tensor(

In [46]:
def Translate(src_sentence, src_field, trg_field, model):
    ip_tensor = ipTensor(src_sentence, src_field)
    max_len = 4*ip_tensor.shape[0]
    src_len = [ip_tensor.shape[0]]
    hidden=(torch.zeros((1, 1, e_h_size)),torch.zeros((1, 1, e_h_size)))
    with torch.no_grad():
        enc_outputs,enc_states = model.enc(ip_tensor,hidden)
    dec_states = enc_states
    sos_id = trg_field.vocab.stoi[trg_field.init_token]
    eos_id = trg_field.vocab.stoi[trg_field.eos_token]
    predicts = [sos_id]
    length = 1
    while length < max_len:
        input = torch.LongTensor([predicts[-1]]).view((1, 1)).to(device)
        with torch.no_grad():
            output,dec_states = model.dec(input, dec_states, enc_outputs)
        dec_states=dec_states.squeeze(0)
        output = output.squeeze()
        output = output.view(-1, model.dec.output_size)
        predicts.append(output.argmax(-1).item())
        length += 1
        if predicts[-1] == eos_id:
            break
    sentence = [trg_field.vocab.itos[id] for id in predicts[1:]]
    return sentence

In [47]:
ind = int(random.random() * len(test_data.examples))
example = test_data.examples[ind]
src_sentence = example.src
trg_sentence = example.trg
print("German: ", ' '.join(src_sentence))
translation = Translate(src_sentence, SRC, TRG, model)
print("English: ", ' '.join(translation[:-1]))
print("Actual Translation: ", ' '.join(trg_sentence))

German:  marathonläuferinnen laufen auf einer städtischen straße während andere personen um sie herum stehen .
English:  firetrucks are around around a street around around while other people around around .
Actual Translation:  marathon runners are racing on a city street , with other people standing around .


In [48]:
def Calculate_BLEU(data, src_field, trg_field, model):
    trgs = []
    predicted_trgs = []
    for i in range(len(data.examples)):
        src_sentence = vars(data[i])['src']
        trg_sentence = vars(data[i])['trg']
        try:                               
            predicted_trg = Translate(src_sentence, src_field, trg_field, model)
            predicted_trgs.append(predicted_trg[:-1])
            trgs.append([trg_sentence])
        except:
            pass
    return bleu_score(predicted_trgs, trgs)

In [49]:
bleu_score_test = Calculate_BLEU(test_data, SRC, TRG, model)
print(f"BLEU score on Testing Data: {bleu_score_test*100:.2f}")

BLEU score on Testing Data: 21.95
